In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
Sales = pd.read_excel(r"D:\Software\SQL Case Studies\Case Study_1\Sales.xlsx")

In [3]:
menu = pd.read_excel(r"D:\Software\SQL Case Studies\Case Study_1\Menus.xlsx")

In [4]:
members = pd.read_excel(r"D:\Software\SQL Case Studies\Case Study_1\Members.xlsx")

In [8]:
Sales

,customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3
5,A,2021-01-11,3
6,B,2021-01-01,2
7,B,2021-01-02,2
8,B,2021-01-04,1
9,B,2021-01-11,1


In [9]:
members

,customer_id,join_date
0,A,2021-01-07
1,B,2021-01-09


In [10]:
menu

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [12]:
sales_menu = Sales.merge(menu , how = "inner")

### 1. What is the total amount each customer spent at the restaurant?

In [198]:
sales_menu
sales_menu.groupby("customer_id", as_index = False)["price"].sum()

,customer_id,price
0,A,76
1,B,74
2,C,36


### 2. How many days has each customer visited the restaurant?

In [197]:
sales_menu
sales_menu.groupby("customer_id")["order_date"].nunique()

customer_id
A    4
B    6
C    2
Name: order_date, dtype: int64

### 3.What was the first item from the menu purchased by each customer?

In [196]:

sales_menu
first_date = sales_menu.groupby("customer_id", as_index = False)["order_date"].min()
first_item = first_date.merge(sales_menu, how ="left")
first_item[["customer_id","product_name"]]

,customer_id,product_name
0,A,sushi
1,A,curry
2,B,curry
3,C,ramen
4,C,ramen


### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [195]:

sales_menu
most_purchased = sales_menu.groupby("product_name", as_index = False)["product_id"].count().sort_values("product_id", ascending = False)
most_purchased.rename(columns = {"product_id":"No_purchased"}, inplace = True)
most_purchased.loc[1]

product_name    ramen
No_purchased        8
Name: 1, dtype: object

### 5. Which item was the most popular for each customer?

In [191]:

sales_menu
No_items = sales_menu.groupby(["customer_id","product_name","product_id"],as_index = False)["product_id"].count().sort_values(["customer_id","product_id"], ascending =(True, False))
No_items.rename(columns = {"product_id":"No_purchased"}, inplace = True)
No_items

,customer_id,product_name,No_purchased
1,A,ramen,3
0,A,curry,2
2,A,sushi,1
3,B,curry,2
4,B,ramen,2
5,B,sushi,2
6,C,ramen,3


### 6. Which item was purchased first by the customer after they became a member?

In [190]:

sales_menu_members = sales_menu.merge(members, how = "inner")
sales_menu_members
after_joining = sales_menu_members[sales_menu_members["order_date"] >= sales_menu_members["join_date"]]
date_demanded = after_joining.groupby("customer_id", as_index =False)["order_date"].min()
item_first_purchased = date_demanded.merge(sales_menu_members, how = "left")
item_first_purchased[["customer_id","product_name","order_date","join_date"]]

,customer_id,product_name,order_date,join_date
0,A,curry,2021-01-07,2021-01-07
1,B,sushi,2021-01-11,2021-01-09


### 7. Which item was purchased just before the customer became a member? 

In [189]:
sales_menu_members
before_joining = sales_menu_members[sales_menu_members["order_date"] < sales_menu_members["join_date"]]
date_demanded_ = before_joining.groupby("customer_id", as_index = False)["order_date"].max()
items_purchased = date_demanded_.merge(sales_menu_members, how = "left")
items_purchased[["customer_id","product_name","order_date","join_date"]]

,customer_id,product_name,order_date,join_date
0,A,sushi,2021-01-01,2021-01-07
1,A,curry,2021-01-01,2021-01-07
2,B,sushi,2021-01-04,2021-01-09


### 8. What is the total items and amount spent for each member before they became a member?

In [188]:
before_members = sales_menu_members[sales_menu_members["order_date"] < sales_menu_members["join_date"]]
total = before_members.groupby("customer_id",as_index = False).agg({"product_id":"count","price":"sum"})
total.rename(columns = {"product_id":"items" , "price":"spent" })

,customer_id,items,spent
0,A,2,25
1,B,3,40


### 9. If each 1 dollar spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [187]:

def condition(table):
    if table["product_name"] == "sushi":
        value = table["price"] * 20
    else:
        value = table["price"] * 10
    return value

sales_menu["points"] = sales_menu.apply(condition, axis = 1)
sales_menu.groupby("customer_id")["points"].sum()


customer_id
A    860
B    940
C    360
Name: points, dtype: int64

### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi -how many points do customer A and B have at the end of January?

In [186]:


sales_menu_members["after_week"] = sales_menu_members["join_date"] + timedelta(days = 7)
def cond(table_2):
    if table_2["order_date"] <= table_2["after_week"]  and table_2["order_date"] >= table_2["join_date"]:
        value_1 = table_2["points"] * 2
    else:
        value_1 = table_2["points"]
    return value_1

sales_menu_members["points_week"] = sales_menu_members.apply(cond , axis = 1S)
sales_menu_members.groupby("customer_id")["points_week"].sum()



customer_id
A    1370
B    1260
Name: points_week, dtype: int64